In [10]:
!pip install pymysql

In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pymysql
import requests
import time
import os
import shutil

In [12]:
def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
        [(0,0),(0,300),(200,0),(400,300),(400,0)]
    ])          # Triangle polygon because cv2.fillPoly expects an array of polygons.
    a=cv2.fillPoly(image,polygons,(255,255,255)) 
    return a

In [13]:

conn = pymysql.connect(host='localhost', user='root', password='1234', charset='utf8', db='capstone0') #DB 연결
cur = conn.cursor()

In [22]:
# clone darknet repo
#!git clone https://github.com/AlexeyAB/darknet

In [23]:
#pip install jupyter

In [24]:
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# train, valid txt 생성


In [25]:
#from glob import glob

#img_list = glob('C:\Users\twan8\Desktop\YOLOtest\darknet-master\build\darknet\x64\data\obj\*.jpg')

#이미지 갯수 출력
#len(img_list)

# cfg 경로
C:\Users\twan8\Desktop\YOLOtest\darknet-master\cfg\yolov4-obj.cfg

# names 경로
C:\Users\twan8\Desktop\YOLOtest\darknet-master\build\darknet\x64\data\obj.names
# data 경로
C:\Users\twan8\Desktop\YOLOtest\darknet-master\build\darknet\x64\data\obj.data
# train 경로
C:\Users\twan8\Desktop\YOLOtest\darknet-master\build\darknet\x64\data\train.txt
# test 경로
C:\Users\twan8\Desktop\YOLOtest\darknet-master\build\darknet\x64\data\test.txt
# 이미지 테스트 경로
C:\Users\twan8\Desktop\YOLOtest\darknet-master\build\darknet\x64\data\obj\gcooter_100001.jpg
# backup 경로
C:\Users\twan8\Desktop\YOLOtest\darknet-master\build\darknet\x64\backup

In [14]:
%cd C:\\Users\\twan8\\Desktop\\YOLOtest\\darknet-master
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
#!sed -i 's/GPU=0/GPU=1/' Makefile
#!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
#!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile


C:\Users\twan8\Desktop\YOLOtest\darknet-master


'sed'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [15]:
!make

'make'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [16]:
# 이미지를 보여주는 함수 정의
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  #%matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [17]:
import socket
import re
host = '192.168.219.86' # 호스트 ip를 적어주세요
port = 4000            # 포트번호를 임의로 설정해주세요

server_sock = socket.socket(socket.AF_INET)
server_sock.bind((host, port))
server_sock.listen(1)

print("기다리는 중")
client_sock, addr = server_sock.accept()

print('Connected by', addr)

기다리는 중
Connected by ('192.168.219.190', 50481)


In [18]:
%cd C:/Users/twan8/Desktop/YOLOtest/darknet-master/build/darknet/x64
origin = "C:\\Users\\twan8\\Desktop\\YOLOtest\\darknet-master\\build\\darknet\\x64\\predictions.jpg"

num = 1
num2= 1
while(1):
    command = "darknet_no_gpu.exe detector test data/obj.data ../../../cfg/yolov4-obj.cfg yolov4-obj_final.weights -thresh 0.50 data/obj/" + str(num) + ".jpg -out detect.txt"
    cap = cv2.VideoCapture('http://192.168.219.117:81/stream')
    _, frame = cap.read()
    #cropped_image = region_of_interest(frame)
    cv2.imshow('Result',frame)
    cv2.imwrite("C:\\Users\\twan8\\Desktop\\YOLOtest\\darknet-master\\build\\darknet\\x64\\data\\obj\\" + str(num) + ".jpg", frame)
    num=num+1
    cv2.destroyAllWindows()
    os.system(command)

    with open('detect.txt') as f:
        if 'Gcooter' in f.read():
            tmp = "C:\\Users\\twan8\\eclipse-workspace\\Web\\WebContent\\resources\\images\\"+str(num2)+".jpg"
            shutil.copy(origin,tmp)
            
            data = client_sock.recv(1024)
            #print(data.decode("utf-8"))
            data = data.decode("utf-8")
            
            numbers = re.findall("\d", data)
            la=''
            lo=''
            end = 9
            if len(numbers)==20:
                end = 10 
            for i in range(end):
                la+=numbers[i]
            la = float(int(la)/10000000)
            for i in range(10,0,-1):
                lo += numbers[-i]
            lo = float(int(lo)/10000000)
            print(la,"   ",lo)

            sql = "INSERT INTO result VALUES("+str(num2)+","+str(la)+","+str(lo)+")"
            cur.execute(sql)
            conn.commit()
            requests.get('http://localhost:8080/Web/testResult.jsp?dectect=ok')
            
            print("True")

            num2=num2+1
        else:
            print("false")

C:\Users\twan8\Desktop\YOLOtest\darknet-master\build\darknet\x64
35.1401399     126.93388
True


ConnectionAbortedError: [WinError 10053] 현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다

In [ ]:
#%cd C:/Users/twan8/Desktop/YOLOtest/darknet-master/build/darknet/x64

#!darknet_no_gpu.exe detector test data/obj.data ../../../cfg/yolov4-obj.cfg yolov4-obj_final.weights -thresh 0.25 data/obj/aaa.jpg -out detect.txt

In [ ]:
# show image using our helper function
#%cd C:/Users/twan8/Desktop/YOLOtest/darknet-master/build/darknet/x64
#imShow('predictions.jpg')